In [1]:
import os
import json
import copy
import numpy as np
import pandas as pd

In [2]:
rewards_per_ep = pd.read_csv('data/reward_log.txt',header=None)

In [3]:
rewards_per_ep = rewards_per_ep[0].apply(lambda x: int(x.split()[1]))

In [4]:
dfs = []
for exp_f in os.listdir('data'):
    if not exp_f.startswith('expert') or not exp_f.endswith('.txt'):
        continue

    print('reading',exp_f)
    f = open(f'data/{exp_f}')
    data = json.load(f)
    f.close()

    df = []
    for k, v in data.items():
        o, a = v
        o.append(a)
        df.append(o)

    df = pd.DataFrame(df).iloc[:,:129]
    dfs.append(df)

reading expert_2022-04-18T15-08-29.txt
reading expert_2022-04-18T15-22-57.txt
reading expert_2022-04-18T15-26-55.txt
reading expert_2022-04-18T15-31-12.txt
reading expert_2022-04-18T15-34-37.txt
reading expert_2022-04-18T15-37-46.txt
reading expert_2022-04-18T15-40-20.txt
reading expert_2022-04-18T15-43-02.txt
reading expert_2022-04-18T15-46-45.txt
reading expert_2022-04-18T15-49-47.txt
reading expert_2022-04-18T17-45-40.txt
reading expert_2022-04-18T17-52-24.txt
reading expert_2022-04-18T17-54-38.txt
reading expert_2022-04-18T17-58-00.txt
reading expert_2022-04-18T18-00-32.txt
reading expert_2022-04-18T18-03-22.txt
reading expert_2022-04-18T18-05-46.txt
reading expert_2022-04-18T18-08-26.txt
reading expert_2022-04-18T18-11-25.txt
reading expert_2022-04-18T18-14-07.txt


In [5]:
pd.concat(dfs).isna().sum().value_counts()

0    129
dtype: int64

In [6]:
train = pd.concat(dfs)
train.head()

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
0,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,2,66,240,146,215,3
1,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,2,66,240,146,215,3
2,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,2,66,240,146,215,3
3,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,2,66,240,146,215,3
4,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,2,66,240,146,215,3


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23484 entries, 0 to 1177
Columns: 129 entries, 0 to 128
dtypes: int64(129)
memory usage: 23.3 MB


In [8]:
X = train.drop(128, axis=1)
y = train[128]

In [9]:
y = pd.get_dummies(y)
y.insert(0, 0, 0)
y.insert(5, 5, 0)
y.insert(6, 6, 0)
y.insert(7, 7, 0)
y.insert(8, 8, 0)

In [10]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,0,2,66,240,146,215
1,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,0,2,66,240,146,215
2,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,0,2,66,240,146,215
3,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,0,2,66,240,146,215
4,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,0,2,66,240,146,215


In [11]:
y.head()

,0,1,2,3,4,5,6,7,8
0,0,0,0,1,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0


In [12]:
X.shape

(23484, 128)

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [15]:
class DDQN(nn.Module):
    """
    The Double Deep Q-Network has as input a state s and
    outputs the state-action values q(s,a_1), ..., q(s,a_n) for all n actions.
    :param: state_dim: for input layer
    :param: hidden_dim: for every hidden layer
    :param: action_dim: for output layer
    """
    def __init__(self, action_dim, state_dim, hidden_dim):
        super().__init__()

        self.online = nn.Sequential(
            nn.Linear(state_dim, hidden_dim*2),
            nn.LeakyReLU(),
            nn.Linear(hidden_dim*2, hidden_dim),
            nn.LeakyReLU(),
            nn.Linear(hidden_dim, action_dim)
        )

        self.target = copy.deepcopy(self.online)

        # Q_target parameters are frozen.
        for p in self.target.parameters():
            p.requires_grad = False

    def forward(self, input, model):
        """
        When doing update by forward, it takes:
        :param: input: all state of each observation
        :param: model: online or target
        :return: Q_values of all actions given state from online/target
        """

        if model == "online":
            return self.online(input)
        elif model == "target":
            return self.target(input)

In [31]:
def train(model, epochs, X_train, X_test, y_train, y_test, batch_size=32):

    model = model
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.00001)
    
    train_loss = [loss_fn(model(X_train, 'online'), y_train).item()]
    print(f'epoch: {0}, train loss = {train_loss[0]: .4f}')
    
    running_loss = 0.0
    for epoch in range(epochs):
        
        # random the order of X_train and y_train for batching
        permutation = torch.randperm(X_train.size()[0])
        
        avg_train_loss = []
        avg_test_loss = []
        
        for i in range(0, X_train.size()[0], batch_size):
            
            indices = permutation[i:i+batch_size]
            batch_x, batch_y = X_train[indices], y_train[indices]
            
            # init optimizer
            optimizer.zero_grad()

            # forward pass and loss
            batch_y_hat = model(batch_x, 'online')
            loss = loss_fn(batch_y_hat, batch_y)

            # backward pass
            loss.backward()

            # update
            optimizer.step()
            
            avg_train_loss.append(loss.item())
        
        train_loss.append(np.mean(avg_train_loss))
        y_pred = model(X_test, 'online')
        
        val_loss = loss_fn(y_pred, y_test)
        avg_test_loss.append(np.mean(val_loss.item()))

        if (epoch + 1) % 10 == 0:
            print(f'epoch: {epoch+1}, train loss = {loss.item(): .4f}, test loss = {val_loss.item(): .4f}')
    
    return avg_train_loss, avg_test_loss

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, stratify=y)

In [21]:
X_train = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train.values, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test.values, dtype=torch.float32).to(device)

Training

In [29]:
net = DDQN(9,128,128).to(device)

retrain = False # True
if retrain:
    checkpoint = torch.load("data/expert_CE.chkpt", map_location=torch.device('cpu'))
    net.load_state_dict(checkpoint['model'])

In [32]:
train(net, 50, X_train, X_test, y_train, y_test)

epoch: 0, train loss =  0.8282
epoch: 10, train loss =  0.9480, test loss =  0.9155
epoch: 20, train loss =  0.7107, test loss =  0.8950
epoch: 30, train loss =  0.7097, test loss =  0.8696
epoch: 40, train loss =  0.7362, test loss =  0.8583
epoch: 50, train loss =  0.5293, test loss =  0.8453


([0.6679730415344238,
  0.718706488609314,
  0.5056966543197632,
  0.6972368955612183,
  0.5345399975776672,
  0.6854649782180786,
  0.6328055262565613,
  0.5927356481552124,
  0.8564898371696472,
  0.5288715362548828,
  0.7811293601989746,
  0.7239629626274109,
  0.6966859698295593,
  0.7225309014320374,
  0.6797051429748535,
  0.7152587175369263,
  0.5407190322875977,
  0.757960319519043,
  0.6365983486175537,
  0.6549027562141418,
  0.6250765323638916,
  0.44561904668807983,
  0.6260679364204407,
  0.4876243472099304,
  0.5606369972229004,
  0.4195913076400757,
  0.678602933883667,
  0.5023676156997681,
  0.5965218544006348,
  0.5387930274009705,
  0.6152819991111755,
  0.6889351606369019,
  0.6937064528465271,
  0.49560749530792236,
  0.3879130780696869,
  0.47004279494285583,
  0.8159167766571045,
  0.776059627532959,
  0.5666105151176453,
  0.4622281789779663,
  0.5599296689033508,
  0.8033310174942017,
  0.7847638130187988,
  0.5619091391563416,
  0.7469055652618408,
  0.7437815

In [27]:
X = torch.tensor(X.values, dtype=torch.float32).to(device)
y = torch.tensor(y.values, dtype=torch.float32).to(device)

In [34]:
loss_fn = nn.CrossEntropyLoss()
y_pred = net(X, 'online')
loss_fn(y_pred, y).item()

0.7201058268547058

In [33]:
# save trained weight

net.target.load_state_dict(net.online.state_dict())
torch.save(dict(model=net.state_dict(), exploration_rate=1.0), "data/expert_CE_1.chkpt")

Checking

In [20]:
model = DDQN(9,128,128).to(device)
loss_fn = nn.SmoothL1Loss()
checkpoint = torch.load("data/expert1.chkpt", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model'])
y_pred = model(X, 'online')
loss_fn(y_pred, y).item()

0.05048644542694092

In [21]:
model = DDQN(9,128,128).to(device)
loss_fn = nn.SmoothL1Loss()
checkpoint = torch.load("data/expert2.chkpt", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model'])
y_pred = model(X, 'online')
loss_fn(y_pred, y).item()

0.02409677766263485

In [22]:
model = DDQN(9,128,128).to(device)
loss_fn = nn.SmoothL1Loss()
checkpoint = torch.load("data/expert4.chkpt", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model'])
y_pred = model(X, 'online')
loss_fn(y_pred, y).item()

0.013653799891471863

In [27]:
model = DDQN(9,128,128).to(device)
loss_fn = nn.SmoothL1Loss()
checkpoint = torch.load("data/expert5.chkpt", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model'])
y_pred = model(X, 'online')
loss_fn(y_pred, y).item()

0.008327632211148739